### Authenticate GEE and start Geemap

In [ ]:
import ee
import geemap
!pip install geemap


ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=lG6R9VfqBiYHd2qff97WWPpRZP3_z16GEfkdqH6pcso&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6KabJJgBM8R5WqcCKSSa2k3bOVMssd4Jeb7BiF7w-2nc6kdDrW36w

Successfully saved authorization token.


### outline plot

In [ ]:
# Make an area of interest geometry centered on San Francisco.
gulf_coast = ee.Geometry.BBox(-98, 26,-82,31)

# Import a Landsat 8 image, subset the thermal band, and clip to the

# Define a map centered on Gulf Coast
Map = geemap.Map()

Map.setCenter(-90,28,7)
Map.addLayer(gulf_coast, {'color': 'red'}, 'bounding box')
Map.addLayerControl() 
Map

### load delta binary map

In [ ]:
merit = ee.Image("MERIT/DEM/v1_0_3").clip(gulf_coast)#.reduceResolution({reducer: ee.Reducer.mean(), maxPixels: 1024}).reproject({  crs: modisProjection  });

Map = geemap.Map()
dmask = merit.lt(5).selfMask()
Map.setCenter(-90,28,7)
Map.addLayer(dmask)

Map

## label deltas

Labeling objects is often the first step in object analysis. Here, the `connectedComponents()` function is used to identify image objects and assign a unique ID to each; all pixels belonging to an object are assigned the same integer ID value. The result is a copy of the input image with an additional "labels" band associating pixels with an object ID value based on connectivity of pixels in the first band of the image.

In [ ]:
# Uniquely label the hotspot image objects.
object_id = dmask.connectedComponents(connectedness=ee.Kernel.plus(1), maxSize=1024)

Map = geemap.Map()
Map.setCenter(-90,28,7)

# Add the uniquely ID'ed objects to the map.
Map.addLayer(object_id.randomVisualizer(), {}, 'Objects')

# Compute the number of pixels in each object defined by the "labels" band.
object_size = object_id.select('labels').connectedPixelCount()

# Get a pixel area image.
pixel_area = ee.Image.pixelArea()

# Multiply pixel area by the number of pixels in an object to calculate
# the object area. The result is an image where each pixel
# of an object relates the area of the object in m^2.
object_area = object_size.multiply(pixel_area)

minPixelCount = ee.Image(1e8).divide(pixel_area)
object_id = object_id.updateMask(object_size.gte(minPixelCount))

# Add the object area to the map.
# Map.addLayer(object_id) #,{'min': 0, 'max': 5e9, 'palette': ['0000FF', 'FF00FF']},'Object area m^2')

#only add deltas larger than 100km2
area_mask = object_area.gte(1e8)

# Update the mask of the `object_id` layer defined previously using the
# minimum area mask just defined.
# object_id = object_id.updateMask(area_mask)
#Map.addLayer(object_id, {}, 'Large hotspots')


delta = ee.Image(0).addBands(object_id.select('labels'))

# Calculate the mean temperature per object defined by the previously added
# "labels" band.
delta_comp = delta.reduceConnectedComponents(
    reducer=ee.Reducer.mean(), labelBand='labels')

# Add object mean temperature to the map and display it.
#Map.addLayer(delta_comp,{'min': 0, 'max': 5, 'palette': ['yellow', 'red']},'Mean temperature')
Map

## Object size

### Number of pixels

Calculate the number of pixels composing objects using the `connectedPixelCount()` image method. Knowing the number of pixels in an object can be helpful for masking objects by size and calculating object area. The following snippet applies `connectedPixelCount()` to the "labels" band of the `objectId` image defined in the previous section.

In [ ]:
Map = geemap.Map()
Map.setCenter(-90,28,7)

GlobalDeltaData = ee.FeatureCollection('users/jhnienhuis/GlobalDeltaBasins_app_shp');
DeltaMouth = ee.Geometry.MultiPoint(GlobalDeltaData.aggregate_array("MouthLon").zip(GlobalDeltaData.aggregate_array("MouthLat")))

#print(GlobalDeltaData.first())

Map.addLayer(DeltaMouth)
Map